<a href="https://colab.research.google.com/github/shalvamist/GPTme/blob/main/Examples/notebooks/Running_Ollama_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running Ollama in Colab
#### Main credit goes to Gruff and his post on StackOverflow -

[https://stackoverflow.com/questions/77697302/how-to-run-ollama-in-google-colab]
#### Some modification were made to make ollama run locally on Colab

### You will need to sign up to Ngrok and get a key - IT'S FREE for Devs



# Installing Ngrok & Ollama


In [1]:
!pip install pyngrok
!sudo apt-get install lshw
# Download and run the Ollama Linux install script
!curl -fsSL https://ollama.com/install.sh | sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
lshw is already the newest version (02.19.git.2021.06.19.996aaad9c7-2build1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


# Importing packages & defining util functions

In [1]:
import threading
import time
import os
import asyncio
from pyngrok import ngrok
import threading
import queue
import time
from threading import Thread

# Get your ngrok token from your ngrok account:
# https://dashboard.ngrok.com/get-started/your-authtoken
token="YOUR NGROK KEY GOES HERE"
ngrok.set_auth_token(token)

# set up a stoppable thread (not mandatory, but cleaner if you want to stop this later
class StoppableThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(StoppableThread, self).__init__(*args, **kwargs)
        self._stop_event = threading.Event()

    def stop(self):
        self._stop_event.set()

    def is_stopped(self):
        return self._stop_event.is_set()

def start_ngrok(q, stop_event):
    try:
        # Start an HTTP tunnel on the specified port
        public_url = ngrok.connect(11434)
        # Put the public URL in the queue
        q.put(public_url)
        # Keep the thread alive until stop event is set
        while not stop_event.is_set():
            time.sleep(1)  # Adjust sleep time as needed
    except Exception as e:
        print(f"Error in start_ngrok: {e}")

# Opening the Ngrok tunnel

In [2]:
# Create a queue to share data between threads
url_queue = queue.Queue()

# Start ngrok in a separate thread
ngrok_thread = StoppableThread(target=start_ngrok, args=(url_queue, StoppableThread.is_stopped))
ngrok_thread.start()

# Wait for the ngrok tunnel to be established
while True:
    try:
        public_url = url_queue.get()
        if public_url:
            break
        print("Waiting for ngrok URL...")
        time.sleep(1)
    except Exception as e:
        print(f"Error in retrieving ngrok URL: {e}")

print("Ngrok tunnel established at:", public_url)

Error in start_ngrok: 'function' object has no attribute 'is_set'
Ngrok tunnel established at: NgrokTunnel: "https://3318-34-121-79-114.ngrok-free.app" -> "http://localhost:11434"


# Defining util functions for async execution of Ollama

In [3]:
import os
import asyncio

# NB: You may need to set these depending and get cuda working depending which backend you are running.
# Set environment variable for NVIDIA library
# Set environment variables for CUDA
os.environ['PATH'] += ':/usr/local/cuda/bin'
# Set LD_LIBRARY_PATH to include both /usr/lib64-nvidia and CUDA lib directories
# os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia:/usr/local/cuda/lib64'

async def run_process(cmd):
    print('>>> starting', *cmd)
    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    # define an async pipe function
    async def pipe(lines):
        async for line in lines:
            print(line.decode().strip())

        await asyncio.gather(
            pipe(process.stdout),
            pipe(process.stderr),
        )

    # call it
    await asyncio.gather(pipe(process.stdout), pipe(process.stderr))

# Running Ollama Server - takes a few seconds

In [4]:
import asyncio
import threading

async def start_ollama_serve():
    await run_process(['ollama', 'serve'])

def run_async_in_thread(loop, coro):
    asyncio.set_event_loop(loop)
    loop.run_until_complete(coro)
    loop.close()

# Create a new event loop that will run in a new thread
new_loop = asyncio.new_event_loop()

# Start ollama serve in a separate thread so the cell won't block execution
thread = threading.Thread(target=run_async_in_thread, args=(new_loop, start_ollama_serve()))
thread.start()

>>> starting

# Pulling Ollama Mistral - This might take a few min

In [5]:
async def ollama_pull(model="mistral:instruct"):
    await run_process(['ollama', 'pull', model])

def run_async_in_thread(loop, coro):
    asyncio.set_event_loop(loop)
    loop.run_until_complete(coro)
    loop.close()

# Create a new event loop that will run in a new thread
new_loop = asyncio.new_event_loop()

# Start ollama serve in a separate thread so the cell won't block execution
thread = threading.Thread(target=run_async_in_thread, args=(new_loop, ollama_pull()))
thread.start()

>>> starting

# Test code

In [6]:
!pip install langchain-community

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest
pulling e8a35b5937a5... 100% ▕████████████████▏ 4.1 GB
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB
pulling e6836092461f... 100% ▕████████████████▏   42 B
pulling ed11eda7790d... 100% ▕████████████████▏   30 B
pulling f9b1e3196ecf... 100% ▕████████████████▏  483 B
verifying sha256 digest ⠋ pulling manifest
pulling e8a35b5937a5... 100% ▕████████████████▏ 4.1 GB
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB
pulling e6836092461f... 100% ▕████████████████▏   42 B
pulling ed11eda7790d... 100% ▕████████████████▏   30 B
pulling f9b1e3196ecf... 100% ▕████████████████▏  483 B
verifying sha256 digest ⠙ pulling manifest
pulling e8a35b5937a5... 100% ▕████████████████▏ 4.1 GB
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB
pulling e6836092461f... 100% ▕████████████████▏

In [7]:
from langchain_community.llms import Ollama
llm = Ollama(model="mistral:instruct")
llm.invoke("tell me about Italy")

verifying sha256 digest ⠋ pulling manifest
pulling e8a35b5937a5... 100% ▕████████████████▏ 4.1 GB
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB
pulling e6836092461f... 100% ▕████████████████▏   42 B
pulling ed11eda7790d... 100% ▕████████████████▏   30 B
pulling f9b1e3196ecf... 100% ▕████████████████▏  483 B
verifying sha256 digest ⠙ pulling manifest
pulling e8a35b5937a5... 100% ▕████████████████▏ 4.1 GB
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB
pulling e6836092461f... 100% ▕████████████████▏   42 B
pulling ed11eda7790d... 100% ▕████████████████▏   30 B
pulling f9b1e3196ecf... 100% ▕████████████████▏  483 B
verifying sha256 digest ⠹ pulling manifest
pulling e8a35b5937a5... 100% ▕████████████████▏ 4.1 GB
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB
pulling e6836092461f... 100% ▕████████████████▏   42 B
pulling ed11eda7790d... 100% ▕████████████████▏   30 B
pulling f9b1e3196ecf... 100% ▕████████████████▏  483 B
verifying sha256 digest ⠸ pulling manifest
pul

' Italy is a beautiful and historically rich country located in Southern Europe. It is the third-largest European Union member state by both population and economic output. The country is known for its stunning art, architecture, fashion, food, and vibrant culture.\n\nItaly is bordered by several countries including France, Switzerland, Austria, Slovenia, and Greece. It is composed of 20 regions and has a population of approximately 60 million people. The capital city is Rome, which was once the heart of the powerful Roman Empire that ruled much of Europe for over a thousand years.\n\nItaly is famous for its rich artistic heritage, with many world-renowned masterpieces and historical sites. Some of the most popular attractions include the Colosseum, the Leaning Tower of Pisa, the Sistine Chapel, and the canals of Venice. Italy also has an extensive coastline with beautiful beaches and crystal clear waters.\n\nItalian cuisine is renowned worldwide for its delicious flavors and variety. 


{"function":"print_timings","level":"INFO","line":271,"msg":"generation eval time =    8025.39 ms /   333 runs   (   24.10 ms per token,    41.49 tokens per second)","n_decoded":333,"n_tokens_second":41.49330543521182,"slot_id":0,"t_token":24.100273273273274,"t_token_generation":8025.391,"task_id":0,"tid":"133346343511616","timestamp":1710388227}
{"function":"print_timings","level":"INFO","line":281,"msg":"          total time =    8248.07 ms","slot_id":0,"t_prompt_processing":222.677,"t_token_generation":8025.391,"t_total":8248.068,"task_id":0,"tid":"133346343511616","timestamp":1710388227}
[GIN] 2024/03/14 - 03:50:27 | 200 | 25.272326446s |       127.0.0.1 | POST     "/api/generate"
{"function":"update_slots","level":"INFO","line":1627,"msg":"slot released","n_cache_tokens":346,"n_ctx":2048,"n_past":345,"n_system_tokens":0,"slot_id":0,"task_id":0,"tid":"133346343511616","timestamp":1710388227,"truncated":false}


In [8]:
llm.invoke("tell me about Italy in the Roman empire age")

{"function":"launch_slot_with_data","level":"INFO","line":823,"msg":"slot is processing task","slot_id":0,"task_id":336,"tid":"133346343511616","timestamp":1710388248}
{"function":"update_slots","level":"INFO","line":1796,"msg":"slot progression","n_past":9,"n_prompt_tokens_processed":9,"slot_id":0,"task_id":336,"tid":"133346343511616","timestamp":1710388248}
{"function":"update_slots","level":"INFO","line":1821,"msg":"kv cache rm [p0, end)","p0":9,"slot_id":0,"task_id":336,"tid":"133346343511616","timestamp":1710388248}
{"function":"print_timings","level":"INFO","line":257,"msg":"prompt eval time     =     286.38 ms /     9 tokens (   31.82 ms per token,    31.43 tokens per second)","n_prompt_tokens_processed":9,"n_tokens_second":31.42699509040499,"slot_id":0,"t_prompt_processing":286.378,"t_token":31.819777777777777,"task_id":336,"tid":"133346343511616","timestamp":1710388259}
{"function":"print_timings","level":"INFO","line":271,"msg":"generation eval time =   10422.28 ms /   435 ru

" The Roman Empire age, also known as the Imperial Roman period, covers a vast span of time from approximately 27 BC to AD 476 when Rome was ruled by emperors. Italy, the Italian Peninsula, was the center of this powerful civilization and its political, cultural, social, and economic hub.\n\nDuring this era, Rome grew from a small farming community into an influential world power that spread its influence across Europe, North Africa, and the Middle East through conquests and alliances. The Pax Romana or Roman Peace, a long period of relative stability and prosperity, began in AD 27 and lasted for about two centuries.\n\nRoman society during this time was characterized by a complex system of government, with the Senate and various other bodies advising the emperor on policy matters. Rome's legal system, which influenced Western law significantly, was based on precedent, custom, and laws codified under various emperors.\n\nThe Roman economy was primarily agricultural, with Italy producin